In [2]:
import os
import numpy as np
import pandas as pd
DF=pd.read_excel('../NSMD/PreProcessed/29var/df29.xlsx')
DF['datetime']=DF['Date']+' '+DF['Time']
DF['datetime']=pd.to_datetime(DF['datetime'])
DF.set_index('datetime',inplace=True)
DF.drop(['Date','Time','Duration'],axis=1,inplace=True)
DF.sort_index(inplace=True)
#

print('Number of unique Names:',DF['Name'].nunique())
print('Number of columns:',len(DF.columns))
print('Number of rows:',len(DF))
print('Name of columns:',DF.columns)

lengths=[]
samples=[]
for k in DF.Name.unique().tolist():
    lengths.append(len(DF[DF['Name']==k]))
    if len(DF[DF['Name']==k])>=120:
        temp=DF[DF['Name']==k]
        samples.append(temp.iloc[:120,:])
print('Number of samples:',len(samples))
print('Number of Names:',len(DF.Name.unique().tolist()))

samples=pd.concat(samples)
np_samples=[]
for k in samples.Name.unique().tolist():
    np_samples.append(np.array(samples[samples['Name']==k]))
np_samples=np.array(np_samples)
print('Shape of np_samples:',np_samples.shape)

Number of unique Names: 187
Number of columns: 30
Number of rows: 31257
Name of columns: Index(['Name', 'Sad', 'Guilty', 'Happy', 'Hopeless', 'Anxious', 'Stressed',
       'Overwhelmed', 'Angry', 'Calm', 'Energetic', 'Lonely', 'In pain',
       'Dizzy', 'Nauseous', 'Racing Heart', 'Look forward completing tasks',
       'Satisfied with myself', 'Satisfied with my physical appearance',
       'Cravings', 'Crave_Food', 'Enjoying moment', 'With how many people',
       'Eating', 'Eating_healthy', 'In control', 'Concentrated', 'Worried',
       'WrongDoing', 'Impulsivity'],
      dtype='object')
Number of samples: 180
Number of Names: 187
Shape of np_samples: (180, 120, 30)


In [11]:
def preprocess_univariate(data, target_feature_index, n_lags=5):
    """
    Preprocess the data for univariate time series prediction.
    
    Parameters:
    data (numpy array): The original dataset of shape (samples, timesteps, features).
    target_feature_index (int): The index of the target feature.
    n_lags (int): The number of lagged observations to use for prediction.
    
    Returns:
    X (numpy array): Input features of shape (samples, n_lags, features).
    y (numpy array): Target values of shape (samples, 1).
    """
    n_samples, n_timesteps, n_features = data.shape
    X = []
    y = []
    
    for sample in range(n_samples):
        for t in range(n_lags, n_timesteps):
            X.append(data[sample, t-n_lags:t, target_feature_index])
            y.append(data[sample, t, target_feature_index])
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [15]:
np_samples[0]

array([[48216,     1,     1, ...,     2,     7,     1],
       [48216,     1,     1, ...,     1,     7,     1],
       [48216,     1,     1, ...,     1,     7,     1],
       ...,
       [48216,     1,     3, ...,     3,     7,     1],
       [48216,     1,     1, ...,     1,     7,     1],
       [48216,     1,     1, ...,     1,     7,     1]])

In [16]:
preprocess_univariate(np_samples,1,5)

(array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        ...,
        [5, 4, 2, 5, 4],
        [4, 2, 5, 4, 5],
        [2, 5, 4, 5, 6]]),
 array([1, 1, 1, ..., 5, 6, 4]))